## 前言
    
    drf的Serializer主要有两个功能：
        1.将对象序列化
        2.验证模型数据
        
    
## 序列化

    drf的序列化功能的使用方式与django表单的使用方式十分相似。其功能由类Serializer（序列化器）实现。在序列化器中，开发者只需保持定义的字段与模型的字段一致即可。这是因为serializers.Field中含有一个source选项（或者说参数），它指定当序列化或反序列化时应该匹配相应model的哪个字段或属性。默认情况下，其等于Serializer中的字段名称。例如：name = serializers.CharField(source='name')，此处source默认会等于左边定义的字段名称。开发者可自定义序列化后的字段名称，如：name2 = serializers.CharField(source='name')，这与上面方式匹配的字段是相同的，只是后者将值赋给了name2。
    

    为了更好地理解序列化功能，先来看使用方法，再探究其实现原理。

#### 模型准备
    为方便说明，假设有一个用户模型、角色模型如下：

In [ ]:
class MyUser(models.Model):

    USER_GRADE = [
        (1, '普通用户'),
        (2, 'vip'),
        (3, 'svip')
    ]

    username = models.CharField(max_length=20, unique=True)
    is_active = models.BooleanField(default=True)
    password = models.CharField(max_length=10)
    user_type = models.IntegerField(choices=USER_GRADE, default=1)
    hobby = models.OneToOneField('Profile', on_delete=models.CASCADE, null=True)
    roles = models.ManyToManyField('Role', related_name='members')


class Role(models.Model):

    name = models.CharField(max_length=20)


class Profile(models.Model):

    hobby = models.CharField(max_length=20)
    


### Serializer使用方式
    

#### 1. 对一般字段进行序列化
    
    当想要展示的序列化字段不是关系字段（例如 foreign key）时，按model字段定义相应字段即可。

In [ ]:
class MyUserSerializer(serializers.Serializer):
    
    username = serializers.CharField()
    user_type = serializers.IntegerField()
    

    
# 视图中使用序列化的方式

class ShowInfo(APIView):

# authentication_classes = [UserAuthentication]
# permission_classes = [UserPermission]
# throttle_classes = [UserThrottles]
# parser_classes = [JSONParser, FormParser, MultiPartParser]

def get(self, request):
    user = MyUser.objects.all()
    
    # 只需将要序列化地对象或queryset传入Serializer,并赋值给instance参数，进行实例化即可
    # Manny选项表示instance传入地是一个对象，还是包含多个对象地queryset
    ser = MyUserSerializer(instance=user, many=True)
    
    # 调用.data即可获得一个有序字典对象，包含序列化后地数据。
    print(ser.data)
    return HttpResponse(json.dumps(ser.data))

#### 2. 对关系字段进行序列化
    
    关系字段有些是只读的，有些是可读可写的，可查询文档获取。值得注意的是，使用关系字段时，倘若设置read_only=False, 则需要指定queryset查询集,因为写入数据时，需要进行数据验证，这就得提供一个查询集，用于判断所填数据指向的对象是否存在。
    
    对于关系字段的序列化，分为两种情况：
    1.获取关联模型的某条信息用于展示。
    2.获取关联模型的全部或部分信息。

#### 2.1 获取单条信息
    
    drf提供了多种字段用于获取单条信息，并提供了一些额外功能。在字段选项方面值得注意的是manny选项，当字段蕴含多个对象时，many选项需设为True。比如，roles = serializers.SlugRelatedField(many=True, read_only=True, slug_field='role') 因为一个人通常具有多个角色，因此序列化后roles字段是一个列表，其元素是多个Role对象序列化后得到的结果，此时many = True
    

In [2]:

"""
1.  
    StringRelatedField
    
    该字段会将关系模型的__str__()方法返回的结果作为值。

"""
class Profile(models.Model):

    hobby = models.CharField(max_length=20)

    def __str__(self):
        return self.hobby
    

class MyUserSerializer(serializers.Serializer):
    
    username = serializers.CharField()
    user_type = serializers.IntegerField()
    hobby = serializers.StringRelatedField() # hobby为OneToOneField,因此manny=False，这是many的默认值
    
    
 


"""
2.
    PrimaryKeyRelatedField
    
    该字段会将关系模型的主键作为值。可有read_only控制读写

"""

class MyUserSerializer(serializers.Serializer):
    
    username = serializers.CharField()
    user_type = serializers.IntegerField()
    roles = serializers.PrimaryKeyRelatedField(read_only=True, many=True)
   


    
"""
3.
    HyperlinkedRelatedField
    
    该字段会将关系模型的对应字段值反向生成url。可由read_only控制读写

"""    
    
# 假定有一个api接口用于展示roles,格式为 path(^/api/v1/role/<int:pk>/$, Role.as_view(), name="role")
    
class MyUserSerializer(serializers.Serializer):
    
    username = serializers.CharField()
    user_type = serializers.IntegerField()
    
    # 这里lookup_field指定是Role模型中的字段，所以无需写成"Role.pk"
    roles = serializers.HyperlinkedRelatedField(read_only=False, many=True, view_name='role', lookup_field='pk', queryset=Role.objects.all())
    
    
class ShowInfo(APIView):
    
    def get(self, request):
        user = MyUser.objects.all()
        
        # 当字段中还含有queryset时，应添加context字段，将request加入进去
        ser = MyUserSerializer(instance=user, many=True, context={'request': request})
        return HttpResponse(json.dumps(ser.data))
    
# 则其输出可能为：
    {
        'username': '...',
        'user_type': '...',
        'roles':[
            'http://www.example.com/api/v1/role/1/',
            'http://www.example.com/api/v1/role/2/',
            'http://www.example.com/api/v1/role/3/',
        ]
    }
    
    
    
    
"""
4.
    HyperlinkedIdentityField
    
    该字段与HyperlinkedRelatedField很相似，也会将关系模型的对应字段值反向生成url。但是该字段用于对当前表进行一种标识，或者说进一步说明。因此
    只能用于一对一关系。且只能为read_only。

"""    
    # 参数与HyperlinkedRelatedField相似，需要lookup_field、lookup_url_kwarg以及view_name

#### 2.2 获取部分或全部信息

    需要获取关联模型的多条信息时，则需要将字段定义为Serializer的实例

In [ ]:

# 需要展示的字段只需在RoleSeializer中定义即可，比如，倘若只想展示name字段，则可以去掉id字段的定义
class RoleSerializer(serializers.Serializer):
    
    r_id = serializers.IntegerField(source='pk')
    name = serializers.CharField()


class MyUserSerializer(serializers.Serializer):
    
    username = serializers.CharField()
    user_type = serializers.IntegerField()
    roles = RoleSerializer()

### 更简便的序列化方式 - ModelSerializer

    对于模型的序列化，采用serializer.ModelSerializer是更为高效的方式。
   
    通过该方式进行序列化时，开发者无需自定义模型中已有的字段，这些字段会被自动生成，只需通过元信息中的fields属性，指定要序列化模型中哪些字段。

In [ ]:
class MyUserSerializer(serializers.ModelSerializer):
    
    class Meta:
        
        # fields = '__all__'
        # 表示只序列化"username", "roles"这两个字段
        fields = ['username', 'roles']
            

#### 关系字段的序列化
    
    默认情况下，ModelSerializer中的关系字段会被映射成PrimaryKeyRelatedField, 即意味着上面的roles字段序列化之后显示的是id值


#### 1.序列化关联对象中的所有字段
    
    倘若需要将roles字段关联的对象一起序列化，则可以使用depth属性。该属性制定了序列化的深度，默认情况下为0，即只序列化MyUser模型，其关系字段用id值表示。

In [ ]:
class MyUserSerializer(serializers.ModelSerializer):
    
    class Meta:
        
        # fields = '__all__'
        # 表示只序列化"username", "roles"这两个字段
        fields = ['username', 'roles']
        depth = 1
        

#### 2.序列化关联对象的部分字段
    
    倘若需要将关联对象的部分字段序列化，而不是全部字段。那么可以对该关联字段进行自定义。

In [ ]:
class RoleSerializer(serializers.ModelSerializer):
    
    class Meta:
        model = Role
        # 此处指定需要序列化的字段
        fields = ['name', ...]
        

class MyUserSerializer(serializers.ModelSerializer):
    
    # 自定义roles字段
    roles = RoleSerializer(many=True, read_only=True)
    
    class Meta:
        fields = ['username', 'roles']
        depth = 1
        
        
"""
    默认情况下，关联字read_only=True, 倘若设置read_only=False, 则必须重写create,以及update方法，显式定义如何保存关联对象
"""

#### 反向关联字段
    
    ModelSerializer默认不会生成反向关联字段，需要手动在fields属性中写入related_name，或者是默认生成反向关联名称 <field_name>_set

    

#### Meta类

    在ModelSerializer中，需要通过Meta类属性指定字段的选项。各选项如下：
    
    fields：['name', 'id']  指定序列化的字段
    
    exclude：['name', 'id'] 指定不进行序列化的字段
    
    read_only_field：['id'] 指定只读字段，在模型中，editable=False以及AutoField会自动设为只读，无需再显式定义。
    
    depth: 1  指定序列化深度，默认为0，当为1时表示关联字段的全部字段也进行序列化。
    
    extra_kwargs：{
                "password": {'write_only': True}
              }
              对字段添加额外选项。
    
    
    